# Grid'5000 and FIT/IoT-LAB - IPv6

## Introduction

This example shows how to interact with both platforms in a single experiment.

An IPv6 network is built in IoT-LAB platform, composed of a border sensor and CoAP servers.
A node in Grid'5000 is the client, which uses a CoAP client to read the sensor using its global IPv6 address.

Inspired on: https://www.iot-lab.info/legacy/tutorials/contiki-coap-m3/index.html

In [1]:
from enoslib import *
import logging
import sys

Note: Openstack clients not installed


Configuring logging: save DEBUG to a file and INFO to stdout

In [2]:
log = logging.getLogger()
log.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fileHandler = logging.FileHandler("debug.log", 'a')
fileHandler.setLevel(logging.DEBUG)
fileHandler.setFormatter(formatter)
log.addHandler(fileHandler)

cformat = logging.Formatter("[%(levelname)8s] : %(message)s")
consoleHandler = logging.StreamHandler(sys.stdout)
consoleHandler.setFormatter(cformat)
consoleHandler.setLevel(logging.INFO)
log.addHandler(consoleHandler)

## Getting resources

### IoT-LAB provider configuration: reserve M3 nodes in saclay site

Note: It uses the following M3 images: border-router.iotlab-m3 and er-example-server.iotlab-m3.

More details on how to generate these images in: https://www.iot-lab.info/legacy/tutorials/contiki-coap-m3/index.html

In [3]:
job_name="iotlab_g5k-ipv6"
iotlab_dict = {
    "walltime": "01:00",
    "job_name": job_name,
    "resources": {
        "machines": [
            {
                "roles": ["border_router"],
                "archi": "m3:at86rf231",
                "site": "saclay",
                "number": 1,
                "image": "border-router.iotlab-m3",
            },
            {
                "roles": ["sensor"],
                "archi": "m3:at86rf231",
                "site": "saclay",
                "number": 2,
                "image": "er-example-server.iotlab-m3",
            },
        ]
    },
}
iotlab_conf = IotlabConf.from_dictionary(iotlab_dict)

{
    "job_name": "iotlab_g5k-ipv6",
    "walltime": "01:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "border_router"
                ],
                "image": "border-router.iotlab-m3",
                "archi": "m3:at86rf231",
                "site": "saclay",
                "number": 1
            },
            {
                "roles": [
                    "sensor"
                ],
                "image": "er-example-server.iotlab-m3",
                "archi": "m3:at86rf231",
                "site": "saclay",
                "number": 2
            }
        ]
    }
}


### Grid'5000 provider configuration: reserve nodes in grenoble

In [4]:
g5k_dict = {
    "job_type": "allow_classic_ssh",
    "job_name": job_name,
    "resources": {
        "machines": [
            {
                "roles": ["client"],
                "cluster": "yeti",
                "nodes": 1,
                "primary_network": "default",
                "secondary_networks": [],
            },
        ],
        "networks": [
            {"id": "default", "type": "prod", "roles": ["my_network"], "site": "grenoble"}
        ],
    },
}
g5k_conf = G5kConf.from_dictionnary(g5k_dict)

{'roles': ['client'], 'primary_network': 'default', 'secondary_networks': [], 'cluster': 'yeti', 'nodes': 1}
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "iotlab_g5k-ipv6",
    "job_type": "allow_classic_ssh",
    "key": "/home/donassolo/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "client"
                ],
                "primary_network": "default",
                "secondary_networks": [],
                "cluster": "yeti",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "default",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "grenoble"
            }
        ]
    }
}


### We still need a Static provider to interact with the IoT-LAB frontend machine

In [5]:
import iotlabcli.auth
iotlab_user, _ = iotlabcli.auth.get_user_credentials()

iotlab_frontend_conf = (
    StaticConf()
    .add_machine(
        roles=["frontend"],
        address="saclay.iot-lab.info",
        alias="saclay",
        user=iotlab_user
    )
    .finalize()
)

{
    "resources": {
        "machines": [
            {
                "address": "saclay.iot-lab.info",
                "roles": [
                    "frontend"
                ],
                "alias": "saclay",
                "user": "donassol"
            }
        ],
        "networks": []
    }
}


### IoT-LAB: getting resources

In [6]:
iotlab_provider = Iotlab(iotlab_conf)
iotlab_roles, _ = iotlab_provider.init()
print(iotlab_roles)

{
    "job_name": "iotlab_g5k-ipv6",
    "walltime": "01:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "border_router"
                ],
                "image": "border-router.iotlab-m3",
                "archi": "m3:at86rf231",
                "site": "saclay",
                "number": 1
            },
            {
                "roles": [
                    "sensor"
                ],
                "image": "er-example-server.iotlab-m3",
                "archi": "m3:at86rf231",
                "site": "saclay",
                "number": 2
            }
        ]
    }
}
[    INFO] : Waiting for job id (239624) to be in running state
[    INFO] : Job id (239624) is running
[    INFO] : Finished reserving nodes: hosts [], sensors [<IotlabSensor(roles=['border_router'], address=m3-10.saclay.iot-lab.info, site=saclay, uid=4061)>image=border-router.iotlab-m3)>, <IotlabSensor(roles=['sensor'], address=m3-11.saclay.iot-la

### Grid'5000: getting resources

In [7]:
g5k_provider = G5k(g5k_conf)
g5k_roles, g5knetworks = g5k_provider.init()
print(g5k_roles)

/home/donassolo/enoslib_env/lib/python3.8/site-packages/grid5000/__init__.py:111: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  conf = yaml.load(f)


[    INFO] : Reloading iotlab_g5k-ipv6 from grenoble
[    INFO] : Reloading iotlab_g5k-ipv6 from lille
[    INFO] : Reloading iotlab_g5k-ipv6 from luxembourg
[    INFO] : Reloading iotlab_g5k-ipv6 from lyon
[    INFO] : Reloading iotlab_g5k-ipv6 from nancy
[    INFO] : Reloading iotlab_g5k-ipv6 from nantes
[    INFO] : Reloading iotlab_g5k-ipv6 from rennes
[    INFO] : Reloading iotlab_g5k-ipv6 from sophia
[    INFO] : Submitting {'name': 'iotlab_g5k-ipv6', 'types': ['allow_classic_ssh'], 'resources': "{cluster='yeti'}/nodes=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on grenoble
[ WARNING] : Retrying (Retry(total=4, connect=5, read=4, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /stable/sites/grenoble/jobs/1970419
[    INFO] : Waiting for 1970419 on grenoble [2020-12-22 18:44:33]
[    INFO] : All jobs are Running !
{'client': [Host(address=

### Static: getting resources

In [8]:
frontend_provider = Static(iotlab_frontend_conf)
frontend_roles, _ = frontend_provider.init()
print(frontend_roles)

{'frontend': [Host(address='saclay.iot-lab.info', alias='saclay', user='donassol', keyfile=None, port=None, extra={})]}


## Configuring network connectivity

### Enabling IPv6 on Grid'5000 nodes (https://www.grid5000.fr/w/IPv6)

In [9]:
result=run_command("dhclient -6 br0", roles=g5k_roles)

[WARNING]: No inventory was parsed, only implicit localhost is available


None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on yeti-4.grenoble.grid5000.fr]


[WARNING]: Platform linux on host yeti-4.grenoble.grid5000.fr is using the
discovered Python interpreter at /usr/bin/python, but future installation of
another Python interpreter could change this. See https://docs.ansible.com/ansi
ble/2.9/reference_appendices/interpreter_discovery.html for more information.


ok: [yeti-4.grenoble.grid5000.fr]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on yeti-4.grenoble.grid5000.fr]
changed: [yeti-4.grenoble.grid5000.fr]


In [10]:
result = run_command("ip address show dev br0", roles=g5k_roles)
print(result['ok'])

None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on yeti-4.grenoble.grid5000.fr]
ok: [yeti-4.grenoble.grid5000.fr]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on yeti-4.grenoble.grid5000.fr]
changed: [yeti-4.grenoble.grid5000.fr]
{'yeti-4.grenoble.grid5000.fr': {'stdout': '7: br0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default qlen 1000\n    link/ether 24:6e:96:91:1d:b0 brd ff:ff:ff:ff:ff:ff\n    inet 172.16.19.4/20 brd 172.16.31.255 scope global dynamic br0\n       valid_lft 85077sec preferred_lft 85077sec\n    inet6 2001:660:4406:100:4::4/128 scope global \n       valid_lft forever preferred_lft forever\n    inet6 fe80::266e:96ff:fe91:1db0/64 scope link \n       valid_lft forever preferred_lft forever', 'stderr': ''}}


### Starting tunslip command in frontend.

Redirect tunslip command output to a file to read it later.

In [11]:
iotlab_ipv6_net="2001:660:3207:4c0::"
tun_cmd = "sudo tunslip6.py -v2 -L -a %s -p 20000 %s1/64 > tunslip.output 2>&1" % (iotlab_roles["border_router"][0].alias, iotlab_ipv6_net)
result=run_command(tun_cmd, roles=frontend_roles, asynch=3600, poll=0)

None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on saclay]


[WARNING]: Platform linux on host saclay is using the discovered Python
interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/referen
ce_appendices/interpreter_discovery.html for more information.


ok: [saclay]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on saclay]
changed: [saclay]


### Reseting border router

In [12]:
iotlab_roles["border_router"][0].reset()

[    INFO] : Executing command (reset) on nodes (['m3-10.saclay.iot-lab.info'])


### Get the Border Router IPv6 address from tunslip output

In [13]:
result = run_command("cat tunslip.output", roles=frontend_roles)
print(result['ok'])

None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on saclay]
ok: [saclay]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on saclay]
changed: [saclay]
{'saclay': {'stdout': "slip connected to ``172.16.42.10:20000''\n\n18:46:07 opened tun device ``/dev/tun0''\n0000.000 ifconfig tun0 inet `hostname` mtu 1500 up\n0000.017 ifconfig tun0 add 2001:660:3207:4c0::1/64\n0000.024 ifconfig tun0 add fe80::660:3207:4c0:1/64\n0000.030 ifconfig tun0\n\ntun0: flags=4305<UP,POINTOPOINT,RUNNING,NOARP,MULTICAST>  mtu 1500\n        inet 192.168.5.23  netmask 255.255.255.255  destination 192.168.5.23\n        inet6 fe80::660:3207:4c0:1  prefixlen 64  scopeid 0x20<link>\n        inet6 2001:660:3207:4c0::1  prefixlen 64  scopeid 0x0<global>\n        inet6 fe80::1482:f93a:e647:1c81  prefixl

In [14]:
import re
out = result['ok']['saclay']['stdout']
print(out)
match = re.search(rf'Server IPv6 addresses:\n.+({iotlab_ipv6_net}\w{{4}})', out, re.MULTILINE|re.DOTALL)
br_ipv6 = match.groups()[0]
print("Border Router IPv6 address from tunslip output: %s" % br_ipv6)

slip connected to ``172.16.42.10:20000''

18:46:07 opened tun device ``/dev/tun0''
0000.000 ifconfig tun0 inet `hostname` mtu 1500 up
0000.017 ifconfig tun0 add 2001:660:3207:4c0::1/64
0000.024 ifconfig tun0 add fe80::660:3207:4c0:1/64
0000.030 ifconfig tun0

tun0: flags=4305<UP,POINTOPOINT,RUNNING,NOARP,MULTICAST>  mtu 1500
        inet 192.168.5.23  netmask 255.255.255.255  destination 192.168.5.23
        inet6 fe80::660:3207:4c0:1  prefixlen 64  scopeid 0x20<link>
        inet6 2001:660:3207:4c0::1  prefixlen 64  scopeid 0x0<global>
        inet6 fe80::1482:f93a:e647:1c81  prefixlen 64  scopeid 0x20<link>
        unspec 00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00  txqueuelen 500  (UNSPEC)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

0000.405 *** Address:2001:660:3207:4c0::1 => 2001:0660:3207:04c0
0000.451  Starting 'Border router

### Checking ping from Grid'5000 to border router node

In [15]:
result = run_command("ping6 -c3 %s" % br_ipv6, pattern_hosts="client*", roles=g5k_roles)
print(result['ok'])

None

PLAY [client*] *****************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on yeti-4.grenoble.grid5000.fr]
ok: [yeti-4.grenoble.grid5000.fr]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on yeti-4.grenoble.grid5000.fr]
changed: [yeti-4.grenoble.grid5000.fr]
{'yeti-4.grenoble.grid5000.fr': {'stdout': 'PING 2001:660:3207:4c0::4061(2001:660:3207:4c0::4061) 56 data bytes\n64 bytes from 2001:660:3207:4c0::4061: icmp_seq=1 ttl=55 time=43.4 ms\n64 bytes from 2001:660:3207:4c0::4061: icmp_seq=2 ttl=55 time=23.8 ms\n64 bytes from 2001:660:3207:4c0::4061: icmp_seq=3 ttl=55 time=24.10 ms\n\n--- 2001:660:3207:4c0::4061 ping statistics ---\n3 packets transmitted, 3 received, 0% packet loss, time 6ms\nrtt min/avg/max/mdev = 23.759/30.707/43.378/8.975 ms', 'stderr': ''}}


## Installing and using CoAP clients

### Install aiocoap client and lynx on grid'5000 nodes

In [16]:
with play_on(roles=g5k_roles) as p:
    p.apt(name=["python3-aiocoap", "lynx"], state="present")

[    INFO] : Running playbook /home/donassolo/cs2/enoslib_tests/iotlab_g5k/tmpg4t3qq2s with vars:
{}

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [yeti-4.grenoble.grid5000.fr]

PLAY [all] *********************************************************************

TASK [__calling__ apt] *********************************************************


[WARNING]: Updating cache and auto-installing missing dependency: python-apt


changed: [yeti-4.grenoble.grid5000.fr]

PLAY RECAP *********************************************************************
yeti-4.grenoble.grid5000.fr : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

{'code': 0, 'result': [{'yeti-4.grenoble.grid5000.fr': {'ok': 2, 'failures': 0, 'unreachable': 0, 'changed': 1, 'skipped': 0, 'rescued': 0, 'ignored': 0}}], 'playbook': '/home/donassolo/cs2/enoslib_tests/iotlab_g5k/tmpg4t3qq2s'}


### Grab the CoAP server node’s IPv6 address from the BR’s web interface

In [17]:
result = run_command("lynx -dump http://[%s]" % br_ipv6, roles=g5k_roles)
print(result['ok'])

None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on yeti-4.grenoble.grid5000.fr]
ok: [yeti-4.grenoble.grid5000.fr]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on yeti-4.grenoble.grid5000.fr]
changed: [yeti-4.grenoble.grid5000.fr]
{'yeti-4.grenoble.grid5000.fr': {'stdout': '   Neighbors\nfe80::c471\nfe80::3560\n\n   Routes\n2001:660:3207:4c0::3560/128 (via fe80::3560) 1737s\n2001:660:3207:4c0::c471/128 (via fe80::c471) 1772s', 'stderr': ''}}


### For a CoAP server, GET light sensor

In [18]:
out = result['ok'][g5k_roles["client"][0].address]['stdout']
print(out)
match = re.search(r'fe80::(\w{4})', out, re.MULTILINE|re.DOTALL)
node_uid = match.groups()[0]
print(node_uid)

   Neighbors
fe80::c471
fe80::3560

   Routes
2001:660:3207:4c0::3560/128 (via fe80::3560) 1737s
2001:660:3207:4c0::c471/128 (via fe80::c471) 1772s
c471


In [19]:
result = run_command("aiocoap-client coap://[%s%s]:5683/sensors/light" % (iotlab_ipv6_net, node_uid), roles=g5k_roles)
print(result['ok'])

None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on yeti-4.grenoble.grid5000.fr]
ok: [yeti-4.grenoble.grid5000.fr]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on yeti-4.grenoble.grid5000.fr]
changed: [yeti-4.grenoble.grid5000.fr]
{'yeti-4.grenoble.grid5000.fr': {'stdout': '3', 'stderr': '\n(No newline at end of message)'}}


### GET pressure for the same sensor

In [20]:
result = run_command("aiocoap-client coap://[%s%s]:5683/sensors/pressure" % (iotlab_ipv6_net, node_uid), roles=g5k_roles)
print(result['ok'])

None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on yeti-4.grenoble.grid5000.fr]
ok: [yeti-4.grenoble.grid5000.fr]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on yeti-4.grenoble.grid5000.fr]
changed: [yeti-4.grenoble.grid5000.fr]
{'yeti-4.grenoble.grid5000.fr': {'stdout': '999', 'stderr': '\n(No newline at end of message)'}}


## Clean-up phase

### Stop tunslip in frontend node

In [21]:
result = run_command("pgrep tunslip6 | xargs kill", roles=frontend_roles)

None

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
 [started TASK: Gathering Facts on saclay]
ok: [saclay]

TASK [enoslib_adhoc_command] ***************************************************
 [started TASK: enoslib_adhoc_command on saclay]
changed: [saclay]


### Destroy jobs in testbeds

In [22]:
g5k_provider.destroy()
iotlab_provider.destroy()

[    INFO] : Reloading iotlab_g5k-ipv6 from grenoble
[    INFO] : Reloading 1970419 from grenoble
[    INFO] : Reloading iotlab_g5k-ipv6 from lille
[    INFO] : Reloading iotlab_g5k-ipv6 from luxembourg
[    INFO] : Reloading iotlab_g5k-ipv6 from lyon
[    INFO] : Reloading iotlab_g5k-ipv6 from nancy
[    INFO] : Reloading iotlab_g5k-ipv6 from nantes
[    INFO] : Reloading iotlab_g5k-ipv6 from rennes
[    INFO] : Reloading iotlab_g5k-ipv6 from sophia
[    INFO] : Killing the job (grenoble, 1970419)
[    INFO] : Stopping experiment id (239624)
